In [1]:
import boto3
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.errors import HttpError
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from io import FileIO
from decimal import Decimal
import os
import pickle
import pandas as pd

# this is using login to the API via OAuth which enables private data for your own channel

credentials = None

# token.pickle stores the user's credentials from previously successful logins
if os.path.exists('token.pickle'):
    print('Loading credentials from file...')
    with open('token.pickle', 'rb') as token:
        credentials = pickle.load(token)

if not credentials or not credentials.valid:
    if credentials and credentials.expired and credentials.refresh_token:
        print('Refreshing Access Token...')
        credentials.refresh(Request())
    else:
        print('Fetching new tokens...')
        flow = InstalledAppFlow.from_client_secrets_file(
            'client_secrets.json',
            scopes=[
                'https://www.googleapis.com/auth/youtube.readonly',
                'https://www.googleapis.com/auth/yt-analytics.readonly'
                ]
            )
        
        flow.run_local_server(port=8080, prompt='consent')
        # you receive access and refresh tokens
        # refresh token is to get new acccess tokens
        credentials = flow.credentials

        #save
        with open('token.pickle', 'wb') as f:
            print('Saving credentials for future use...')
            pickle.dump(credentials, f)

# Create YouTube API object
youtubeReporting = build('youtubereporting', 'v1', credentials=credentials)

Loading credentials from file...
Refreshing Access Token...


In [6]:
# Function to download report from YouTube Reporting API
def download_report(youtubeReporting, report_url, local_file):
    request = youtubeReporting.media().download(resourceName='')
    request.uri = report_url
    with FileIO(local_file, mode='wb') as fh:
        downloader = MediaIoBaseDownload(fh, request, chunksize=-1)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
    #         if status:
    #             print('Download %d%%.' % int(status.progress() * 100))
    # print('Download Complete!')

# Function to convert date from YYYYMMDD format to ISO 8601 format
def convert_date(date_series):
    return pd.to_datetime(date_series, format='%Y%m%d').dt.strftime('%Y-%m-%dT%H:%M:%SZ')

# Function to upload data to DynamoDB
def upload_to_basic_table(df):
    table = dynamodb.Table('channel_basic_a2')
    with table.batch_writer() as batch:
        for index, row in df.iterrows():
            item = {
                'composite_key': row['composite_key'],
                'createTime': row['createTime'],
                'date': str(row['date']),
                'channel_id': row['channel_id'],
                'video_id': row['video_id'],
                'live_or_on_demand': row['live_or_on_demand'],
                'subscribed_status': row['subscribed_status'],
                'country_code': row['country_code'],
                'views': row['views'],
                'comments': row['comments'],
                'likes': row['likes'],
                'dislikes': row['dislikes'],
                'shares': row['shares'],
                'watch_time_minutes': Decimal(row['watch_time_minutes']).quantize(Decimal('0.01')),
                'average_view_duration_seconds': Decimal(row['average_view_duration_seconds']).quantize(Decimal('0.01')),
                'average_view_duration_percentage': Decimal(row['average_view_duration_percentage']).quantize(Decimal('0.01')),
                'annotation_impressions': row['annotation_impressions'],
                'annotation_clickable_impressions': row['annotation_clickable_impressions'],
                'annotation_clicks': row['annotation_clicks'],
                'annotation_click_through_rate': row['annotation_click_through_rate'],
                'annotation_closable_impressions': row['annotation_closable_impressions'],
                'annotation_closes': row['annotation_closes'],
                'annotation_close_rate': row['annotation_close_rate'],
                'card_teaser_impressions': row['card_teaser_impressions'],
                'card_teaser_clicks': row['card_teaser_clicks'],
                'card_teaser_click_rate': row['card_teaser_click_rate'],
                'card_impressions': row['card_impressions'],
                'card_clicks': row['card_clicks'],
                'card_click_rate': row['card_click_rate'],
                'subscribers_gained': row['subscribers_gained'],
                'subscribers_lost': row['subscribers_lost'],
                'videos_added_to_playlists': row['videos_added_to_playlists'],
                'videos_removed_from_playlists': row['videos_removed_from_playlists'],
                'red_views': row['red_views'],
                'red_watch_time_minutes': Decimal(row['red_watch_time_minutes']).quantize(Decimal('0.01'))
            }
            batch.put_item(Item=item)

def upload_to_combined_table(df):
    table = dynamodb.Table('channel_combined_a2')
    with table.batch_writer() as batch:
        for index, row in df.iterrows():
            item = {
                'composite_key': row['composite_key'],
                'createTime': row['createTime'],
                'date': str(row['date']),
                'channel_id': row['channel_id'],
                'video_id': row['video_id'],
                'live_or_on_demand': row['live_or_on_demand'],
                'subscribed_status': row['subscribed_status'],
                'country_code': row['country_code'],
                'playback_location_type': row['playback_location_type'],
                'traffic_source_type': row['traffic_source_type'], 
                'device_type': row['traffic_source_type'], 
                'operating_system': row['traffic_source_type'], 
                'views': row['traffic_source_type'],
                'watch_time_minutes': Decimal(row['traffic_source_type']), 
                'average_view_duration_seconds': Decimal(row['traffic_source_type']).quantize(Decimal('0.01')),
                'average_view_duration_percentage': Decimal(row['traffic_source_type']).quantize(Decimal('0.01')), 
                'red_views': row['traffic_source_type'],
                'red_watch_time_minutes': Decimal(row['traffic_source_type']).quantize(Decimal('0.01'))
            }
            batch.put_item(Item=item)

# Function to upload data to DynamoDB
def upload_to_demo_table(df):
    table = dynamodb.Table('channel_demographics_a1')
    with table.batch_writer() as batch:
        for index, row in df.iterrows():
            item = {
                'composite_key': row['composite_key'],
                'createTime': row['createTime'],
                'date': str(row['date']),
                'channel_id': row['channel_id'],
                'video_id': row['video_id'],
                'live_or_on_demand': row['live_or_on_demand'],
                'subscribed_status': row['subscribed_status'],
                'country_code': row['country_code'],
                'age_group': row['age_group'],
                'gender': row['gender'],
                'views_percentage': Decimal(row['views_percentage']).quantize(Decimal('0.01'))
            }
            batch.put_item(Item=item)

# Function to upload data to DynamoDB
def upload_to_sharing_table(df):
    table = dynamodb.Table('channel_sharing_service_a1')
    with table.batch_writer() as batch:
        for index, row in df.iterrows():
            item = {
                'composite_key': row['composite_key'],
                'createTime': row['createTime'],
                'date': str(row['date']),
                'channel_id': row['channel_id'],
                'video_id': row['video_id'],
                'live_or_on_demand': row['live_or_on_demand'],
                'subscribed_status': row['subscribed_status'],
                'country_code': row['country_code'],
                'sharing_service': row['sharing_service'],
                'shares': row['shares'],
            }
            batch.put_item(Item=item)
            

# upload a dictionary from the jobs.reports().list() query to the reports table
def upload_to_jobs_table(report):
   table = dynamodb.Table('reports')
   with table.batch_writer() as batch:
      batch.put_item(Item=report)

In [40]:
# Retrieve reports from YouTube Reporting API - basic report
reports_result = youtubeReporting.jobs().reports().list(jobId='a7f41b3e-2b81-488d-8ed0-1f8c236e1a54').execute()

table_name = 'channel_basic_a2'

dynamodb = boto3.resource('dynamodb')

available_report_ids = [report['id'] for report in reports_result['reports']]

# check the reports table in DynamoDB whether the report has already been uploaded
# Define the keys you want to retrieve
ids_to_retrieve = [{'id': key} for key in available_report_ids]

# Perform batch get item operation
response = dynamodb.batch_get_item(
    RequestItems={
        'reports': {
            'Keys': ids_to_retrieve
        }
    }
)

# Check if any items are returned
if 'Responses' in response:
    items = response['Responses']['reports']
    existing_keys = [item['id'] for item in items]
else:
    print("No items returned for the specified keys.")
    existing_keys = []

# create set difference between available reports and those already processed
new_reports = set(available_report_ids) - set(existing_keys)

# Create a new column containing the composite key
composite_key_cols = ['date', 'channel_id', 'video_id', 'live_or_on_demand', 'subscribed_status', 'country_code']

# Iterate through each report and process
for report in reports_result['reports']:
    if report['id'] in new_reports:
        local_file = f"reports/{report['id']}.csv"
        download_report(youtubeReporting, report['downloadUrl'], local_file)
        with open(local_file, 'r') as file:
            df = pd.read_csv(local_file)
            if not df.empty:
                df['createTime'] = report['createTime']
                df['composite_key'] = df[composite_key_cols].astype(str).agg('_'.join, axis=1)
                df['date'] = convert_date(df['date'])
                upload_to_basic_table(df)
                upload_to_jobs_table(report)
        # Delete the file
        os.remove(local_file)

Existing keys: []


In [7]:
# Retrieve reports from YouTube Reporting API - channel demographics
reports_result = youtubeReporting.jobs().reports().list(jobId='4a7e6f19-e49f-4418-9800-f0ba979a8437').execute()

table_name = 'channel_demographics_a1'

dynamodb = boto3.resource('dynamodb')

available_report_ids = [report['id'] for report in reports_result['reports']]

# check the reports table in DynamoDB whether the report has already been uploaded
# Define the keys you want to retrieve
ids_to_retrieve = [{'id': key} for key in available_report_ids]

# Perform batch get item operation
response = dynamodb.batch_get_item(
    RequestItems={
        'reports': {
            'Keys': ids_to_retrieve
        }
    }
)

# Check if any items are returned
if 'Responses' in response:
    items = response['Responses']['reports']
    existing_keys = [item['id'] for item in items]
else:
    print("No items returned for the specified keys.")
    existing_keys = []

# create set difference between available reports and those already processed
new_reports = set(available_report_ids) - set(existing_keys)

# Create a new column containing the composite key
composite_key_cols = ['date', 'channel_id', 'video_id', 'live_or_on_demand', 'subscribed_status', 'country_code', 'age_group', 'gender']

# Iterate through each report and process
for report in reports_result['reports']:
    if report['id'] in new_reports:
        local_file = f"reports/{report['id']}.csv"
        download_report(youtubeReporting, report['downloadUrl'], local_file)
        with open(local_file, 'r') as file:
            df = pd.read_csv(file)
            if not df.empty:
                df['createTime'] = report['createTime']
                df['composite_key'] = df[composite_key_cols].astype(str).agg('_'.join, axis=1)
                df['date'] = convert_date(df['date'])
                upload_to_demo_table(df)
                upload_to_jobs_table(report)
        # Delete the file
        os.remove(local_file)

Existing keys: []


In [5]:
# Retrieve reports from YouTube Reporting API - channel sharing
reports_result = youtubeReporting.jobs().reports().list(jobId='bff80780-0f0f-4caa-af3e-de968ec64e9e').execute()

table_name = 'channel_sharing_service_a1'

dynamodb = boto3.resource('dynamodb')

available_report_ids = [report['id'] for report in reports_result['reports']]

# check the reports table in DynamoDB whether the report has already been uploaded
# Define the keys you want to retrieve
ids_to_retrieve = [{'id': key} for key in available_report_ids]

# Perform batch get item operation
response = dynamodb.batch_get_item(
    RequestItems={
        'reports': {
            'Keys': ids_to_retrieve
        }
    }
)

# Check if any items are returned
if 'Responses' in response:
    items = response['Responses']['reports']
    existing_keys = [item['id'] for item in items]
else:
    print("No items returned for the specified keys.")
    existing_keys = []

# create set difference between available reports and those already processed
new_reports = set(available_report_ids) - set(existing_keys)

# Create a new column containing the composite key
composite_key_cols = ['date', 'channel_id', 'video_id', 'live_or_on_demand', 'subscribed_status', 'country_code', 'sharing_service']

# Iterate through each report and process
for report in reports_result['reports']:
    if report['id'] in new_reports:
        local_file = f"reports/{report['id']}.csv"
        download_report(youtubeReporting, report['downloadUrl'], local_file)
        with open(local_file, 'r') as file:
            df = pd.read_csv(file)
            if not df.empty:
                df['createTime'] = report['createTime']
                df['composite_key'] = df[composite_key_cols].astype(str).agg('_'.join, axis=1)
                df['date'] = convert_date(df['date'])
                upload_to_sharing_table(df)
                upload_to_jobs_table(report)
        # Delete the file
        os.remove(local_file)

Existing keys: []


In [7]:
# Retrieve reports from YouTube Reporting API - combined table
reports_result = youtubeReporting.jobs().reports().list(jobId='82bc9b78-afbf-470e-8c1f-e9a7d2fe280d').execute()

table_name = 'channel_combined_a2'

dynamodb = boto3.resource('dynamodb')

available_report_ids = [report['id'] for report in reports_result['reports']]

# check the reports table in DynamoDB whether the report has already been uploaded
# Define the keys you want to retrieve
ids_to_retrieve = [{'id': key} for key in available_report_ids]

# Perform batch get item operation
response = dynamodb.batch_get_item(
    RequestItems={
        'reports': {
            'Keys': ids_to_retrieve
        }
    }
)

# Check if any items are returned
if 'Responses' in response:
    items = response['Responses']['reports']
    existing_keys = [item['id'] for item in items]
else:
    print("No items returned for the specified keys.")
    existing_keys = []

# create set difference between available reports and those already processed
new_reports = set(available_report_ids) - set(existing_keys)

# Create a new column containing the composite key
composite_key_cols = ['date', 'channel_id', 'video_id', 'live_or_on_demand', 'subscribed_status', 'country_code', 
                       'playback_location_type', 'traffic_source_type', 'device_type', 'operating_system']

# Iterate through each report and process
for report in reports_result['reports']:
    if report['id'] in new_reports:
        local_file = f"reports/{report['id']}.csv"
        download_report(youtubeReporting, report['downloadUrl'], local_file)
        with open(local_file, 'r') as file:
            df = pd.read_csv(file)
            if not df.empty:
                df['createTime'] = report['createTime']
                df['composite_key'] = df[composite_key_cols].astype(str).agg('_'.join, axis=1)
                df['date'] = convert_date(df['date'])
                upload_to_combined_table(df)
                upload_to_jobs_table(report)
        # Delete the file
        os.remove(local_file)

Existing keys: []
